In [1]:
# https://github.com/Bjarten/early-stopping-pytorch/blob/master/pytorchtools.py
# https://blog.csdn.net/weixin_43792166/article/details/97952312
# https://colab.research.google.com/github/PytorchLightning/pytorch-lightning/blob/master/notebooks/01-mnist-hello-world.ipynb#scrollTo=4DNItffri95Q
# https://pytorch-lightning.readthedocs.io/en/latest/lightning_module.html

import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.nn import functional as F

import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt


### Dataset

In [2]:
## load training and test data
train_data = datasets.MNIST(root='./mnist',
                            train = True,
                            transform=torchvision.transforms.ToTensor(),
                            download=True)

test_data = datasets.MNIST(root='./mnist',
                           train = False,
                           transform=torchvision.transforms.ToTensor(),
                           download=True)

In [3]:
## split training data into training and validation\
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(num_train * 0.25))
train_idx, valid_idx = indices[split:], indices[:split]
    

# define samplers for obtaining training and validation batches
from torch.utils.data.sampler import SubsetRandomSampler

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)    

In [4]:
## load data into batches
BATCH_SIZE = 64

train_loader = DataLoader(dataset=train_data, 
                          batch_size=BATCH_SIZE, 
                          sampler=train_sampler,
                          num_workers=8)

validation_loader = DataLoader(dataset=train_data, 
                               batch_size=BATCH_SIZE, 
                               sampler=valid_sampler,
                               num_workers=8)

test_loader = DataLoader(dataset=test_data, 
                         batch_size=BATCH_SIZE, 
                         num_workers=8)


### CNN model

In [5]:
## model

import pytorch_lightning as pl


class CNNClassifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        
        # conv1
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=3,
                stride=1,
                padding=1
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        # conv2
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding=1
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        # flatten
        self.prediction = nn.Linear(32*7*7, 10)
        
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.prediction(x)
        return output
    
    
    def cross_entropy_loss(self, output, labels):
        return F.nll_loss(output, labels)
    
    
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        output = self.forward(x)
        loss = self.cross_entropy_loss(output, y)
        self.log('train_loss', loss)
        return loss
    
    
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        output = self.forward(x)
        loss = self.cross_entropy_loss(output, y)
        self.log('val_loss', loss)
        return loss
    
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer



In [6]:
class MNISTDataModule(pl.LightningDataModule):
    
    def train_dataloader(self):
        return train_loader

    def val_dataloader(self):
        return validation_loader


In [7]:
data_module = MNISTDataModule()
model = CNNClassifier()


In [8]:
# train

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
early_stop_callback = EarlyStopping(monitor='val_loss',
                                    patience=3,
                                    verbose=True)


trainer = pl.Trainer(callbacks=[early_stop_callback])

trainer.fit(model, data_module)

EarlyStopping mode set to min for monitoring val_loss.
GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name       | Type       | Params
------------------------------------------
0 | conv1      | Sequential | 160   
1 | conv2      | Sequential | 4.6 K 
2 | prediction | Linear     | 15.7 K
------------------------------------------
20.5 K    Trainable params
0         Non-trainable params
20.5 K    Total params


1

In [9]:
test_result = trainer.test(model, test_dataloaders=test_loader, verbose=True)

/opt/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: you passed in a test_dataloader but have no test_step. Skipping test loop
  warnings.warn(*args, **kwargs)


In [10]:
test_result

1

In [11]:
model.log

<bound method LightningModule.log of CNNClassifier(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (prediction): Linear(in_features=1568, out_features=10, bias=True)
)>